In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('datafinal.csv')

In [3]:
df.head()

,Unnamed: 0,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_START_MILLISECONDS,IN_BYTES,L4_DST_PORT,L4_SRC_PORT,OUT_BYTES,RETRANSMITTED_OUT_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,TCP_WIN_MIN_IN,TCP_WIN_MIN_OUT,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TOTAL_FLOWS_EXP,LABEL
0,0,0,1616660040010,1616660040010,44,60541,22351,40,0,1024,0,1024,0,1460,0,2293398,2
1,1,0,1616660040068,1616660040068,68,0,0,0,0,0,0,0,0,0,0,2293400,2
2,2,44,1616660040114,1616660040070,189,24650,443,104,0,122,1634,122,1634,0,0,2293404,2
3,3,0,1616660040122,1616660040122,52,8728,61075,0,0,8192,0,8192,0,1440,0,2293407,2
4,4,1,1616660040184,1616660040183,189,443,39762,40,0,502,0,502,0,0,0,2293409,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12207873 entries, 0 to 12207872
Data columns (total 17 columns):
 #   Column                      Dtype
---  ------                      -----
 0   Unnamed: 0                  int64
 1   FLOW_DURATION_MILLISECONDS  int64
 2   FLOW_END_MILLISECONDS       int64
 3   FLOW_START_MILLISECONDS     int64
 4   IN_BYTES                    int64
 5   L4_DST_PORT                 int64
 6   L4_SRC_PORT                 int64
 7   OUT_BYTES                   int64
 8   RETRANSMITTED_OUT_BYTES     int64
 9   TCP_WIN_MAX_IN              int64
 10  TCP_WIN_MAX_OUT             int64
 11  TCP_WIN_MIN_IN              int64
 12  TCP_WIN_MIN_OUT             int64
 13  TCP_WIN_MSS_IN              int64
 14  TCP_WIN_MSS_OUT             int64
 15  TOTAL_FLOWS_EXP             int64
 16  LABEL                       int64
dtypes: int64(17)
memory usage: 1.5 GB


In [5]:
df['LABEL'] = df['LABEL'].astype('category')

In [6]:
df.dtypes

Unnamed: 0                       int64
FLOW_DURATION_MILLISECONDS       int64
FLOW_END_MILLISECONDS            int64
FLOW_START_MILLISECONDS          int64
IN_BYTES                         int64
L4_DST_PORT                      int64
L4_SRC_PORT                      int64
OUT_BYTES                        int64
RETRANSMITTED_OUT_BYTES          int64
TCP_WIN_MAX_IN                   int64
TCP_WIN_MAX_OUT                  int64
TCP_WIN_MIN_IN                   int64
TCP_WIN_MIN_OUT                  int64
TCP_WIN_MSS_IN                   int64
TCP_WIN_MSS_OUT                  int64
TOTAL_FLOWS_EXP                  int64
LABEL                         category
dtype: object

In [7]:
X = df.drop(['LABEL'], axis=1)
y = df['LABEL']

In [ ]:
print("Before, counts of label '3': {}".format(sum(y == 3)))
print("Before, counts of label '2': {}".format(sum(y == 2)))
print("Before, counts of label '1': {}".format(sum(y == 1)))
print("Before, counts of label '0': {} \n".format(sum(y == 0)))

sm = SMOTE(random_state=2, n_jobs=-1)
X_balanced, y_balanced = sm.fit_resample(X, y.ravel())

print("After, counts of label '3': {}".format(sum(y_balanced == 3)))
print("After, counts of label '2': {}".format(sum(y_balanced == 2)))
print("After, counts of label '1': {}".format(sum(y_balanced == 1)))
print("After, counts of label '0': {}".format(sum(y_balanced == 0)))

Before, counts of label '3': 2496814
Before, counts of label '2': 6570058
Before, counts of label '1': 864054
Before, counts of label '0': 2276947 



In [ ]:
import sklearn
from sklearn import metrics, model_selection, tree
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train, X_test_total, y_train, y_test_total = model_selection.train_test_split(X, y, test_size=0.45, random_state=31)

X_test, X_validation, y_test, y_validation = model_selection.train_test_split(X_test_total, y_test_total, test_size=0.33, random_state=31)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_validation = scaler.transform(X_validation)